In [2]:
def keystream(B,S):
    L = B[7] + B[11] + B[30] + B[40] + B[45] + B[54] + B[71]
    Q = B[4]*B[21] + B[9]*B[52] + B[18]*B[37] + B[44]*B[76]
    T = B[5] + B[8]*B[82] + B[34]*B[67]*B[73] + B[2]*B[28]*B[41]*B[65] + B[13]*B[29]*B[50]*B[64]*B[75] \
    + B[6]*B[14]*B[26]*B[32]*B[47]*B[61] + B[1]*B[19]*B[27]*B[43]*B[57]*B[66]*B[78]
    T_c = S[23] + S[3]*S[16] + S[9]*S[13]*B[48] + S[1]*S[24]*B[38]*B[63]
    
    out = L + Q + T + T_c
    return out

def NFSR1_bk_update(S):
    temp = S[30]  + S[1]  +  S[4]  + S[5]  +  S[14] + S[16] + S[17] + S[19] +  S[24] + S[7]*S[17] + S[7]*S[19] + S[11]*S[20] \
        + S[13]*S[18] + S[16]*S[20] + S[19]*S[21] + S[3]*S[11]*S[21] + S[3]*S[18]*S[21] + S[6]*S[19]*S[20] \
        + S[7]*S[17]*S[21] + S[7]*S[19]*S[21] + S[11]*S[18]*S[21] + S[19]*S[20]*S[21] + S[3]*S[6]*S[11]*S[20]\
        + S[3]*S[6]*S[18]*S[20] + S[3]*S[11]*S[20]*S[21] + S[3]*S[18]*S[20]*S[21] + S[6]*S[7]*S[17]*S[20] \
        + S[6]*S[7]*S[19]*S[20] + S[6]*S[11]*S[18]*S[20] + S[7]*S[17]*S[20]*S[21] + S[7]*S[19]*S[20]*S[21]\
        + S[11]*S[18]*S[20]*S[21]
    
    S = [temp] + S[:30]
    return S

def NFSR2_bk_update(B,s0):
    temp = s0 + B[89] + B[23] + B[48] + B[78] + B[83] + B[2]*B[58] + B[9]*B[11] + B[14]*B[15] + B[24]*B[52] \
    + B[34]*B[41] + B[54]*B[57] + B[59]*B[73] + B[19]*B[21]*B[22] + B[61]*B[67]*B[71] \
    + B[76]*B[79]*B[80]*B[82]
    
    B = [temp] + B[:89]
    return B
    
def NFSR1_fw_update(S):
    temp = S[0]  + S[2]  +  S[5]  + S[6]  +  S[15] + S[17] + S[18] + S[20] +  S[25] + S[8]*S[18] + S[8]*S[20] + S[12]*S[21] \
        + S[14]*S[19] + S[17]*S[21] + S[20]*S[22] + S[4]*S[12]*S[22] + S[4]*S[19]*S[22] + S[7]*S[20]*S[21] \
        + S[8]*S[18]*S[22] + S[8]*S[20]*S[22] + S[12]*S[19]*S[22] + S[20]*S[21]*S[22] + S[4]*S[7]*S[12]*S[21]\
        + S[4]*S[7]*S[19]*S[21] + S[4]*S[12]*S[21]*S[22] + S[4]*S[19]*S[21]*S[22] + S[7]*S[8]*S[18]*S[21] \
        + S[7]*S[8]*S[20]*S[21] + S[7]*S[12]*S[19]*S[21] + S[8]*S[18]*S[21]*S[22] + S[8]*S[20]*S[21]*S[22]\
        + S[12]*S[19]*S[21]*S[22]
    
    for i in range(30):
        S[i]=S[i + 1]
    S[30]=temp
    return(S)

    
def NFSR2_fw_update(B,s0):
    temp = s0 + B[0] + B[24] + B[49] + B[79] + B[84] + B[3]*B[59] + B[10]*B[12] + B[15]*B[16] + B[25]*B[53] \
    + B[35]*B[42] + B[55]*B[58] + B[60]*B[74] + B[20]*B[22]*B[23] + B[62]*B[68]*B[72] \
    + B[77]*B[80]*B[81]*B[83]
    
    for i in range(89):
        B[i]=B[i + 1]
    B[89]=temp
    return(B)


def intersection(A,B):
    temp = []
    for i in range(len(A)):
        if A[i] in B:
            temp.append(A[i])
    return temp


In [1]:
filename = 'Lizard_TMDTO_Output.txt'
for keys in range(20,22,1):
    import time
    start2 = time.time()

    state_len = 121
    V = BooleanPolynomialRing(state_len,['b%d'%(i) for i in range(90)] + ['s%d'%(i) for i in range(31)])
    #V.inject_variables()
    g = list(V.gens()) 

    #Assigning variables to the registers
    B = g[:90] 
    S = g[90:]
    
    
    R = [90+i for i in [23,24,25,26,27,28,29]]
    F = [i for i in range(38,44,1)]
    G = [i for i in range(state_len) if i not in R+F]
    for i in F:
        if i < 90:
            B[i] = 0
        else:
            S[i-90] = 0


    linear = []
    linear_all = []
    n1 = 0
    Z = [0]*keys

    for loop in range(keys):
        Z[loop] = keystream(B,S)
        monom = list(Z[loop].monomials())
        lin = []
        for i in monom:
            if i.degree() > 1:
                n1 += 1
            if i.degree() == 1 and g.index(i) not in linear_all:
                lin = lin + [g.index(i)]
        linear.append(lin)
        linear_all = linear_all + lin
        print("loop 1st part ",loop)
        if loop != keys-1:
            B=NFSR2_fw_update(B,S[0])
            S=NFSR1_fw_update(S) 
    print("1st part done")

    #Assigning variables to the registers
    B = g[:90] 
    S = g[90:]

    for i in F:
        if i < 90:
            B[i] = 0
        else:
            S[i-90] = 0



    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    model = gp.Model()


    v = [0,1,2]             # Temporary variable to be used for gurobi variable definition
    aux = [i for i in range(n1)]

    #Let D, F, G represents the class of determined/recovery, Fixed bit, and Guess bit
    #Since these are set and we need variables to denote R,F and G therefore let 2=R, 3=F and 4=G
    #x represents variables for each register and m contains 3 variables for each x[i] i.e. for R,F and G
    x = model.addVars(len(B) + len(S), lb = 2, ub = 4, vtype = GRB.INTEGER, name = "x")
    m = model.addVars(len(B) + len(S), len(v), vtype = GRB.BINARY, name = "m")
    q = model.addVars(len(aux), vtype = GRB.BINARY, name = "q")
    #p = model.addVars(len(aux), vtype = GRB.BINARY, name = "p")
    #r = model.addVars(len(aux), vtype = GRB.BINARY, name = "r")
    model.update()

    model.addConstrs((m[i,0] == 1) for i in R)
    model.addConstrs((m[i,1] == 1) for i in F)
    model.addConstrs((m[i,2] == 1) for i in G)

    #Objective function is to maximize recovery bit and minimize the number of FIXED bit
    model.setObjectiveN(gp.quicksum((m[i,0]) for i in range(state_len)),0,1, GRB.MAXIMIZE)
    model.setObjectiveN(gp.quicksum((m[i,1]) for i in range(state_len)),1,0, GRB.MINIMIZE)
    #model.setObjectiveN(gp.quicksum((m[i,1]) for i in range(state_len)), GRB.MINIMIZE)
    
    
    model.addConstrs((x[i] == (2*m[i,0] + 3*m[i,1] + 4*m[i,2])) for i in range(state_len))
    model.addConstrs(m[i,0] + m[i,1] + m[i,2] == 1 for i in range(state_len))

    #Number of recovered bit should be less than or equal to the number of keystream equations
    #model.addConstr(gp.quicksum(m[i,0] for i in range(state_len)) == keys)
    model.addConstr(gp.quicksum(m[i,0] for i in range(state_len) if i not in linear_all) == 0)

    count = 0
    cnt = 0
    for loop in range(keys):
        z = Z[loop]
        #z_var contains the index of variables in Z
        #z_var = np.array(z.variables())
        #z_var = [g.index(i) for i in z_var]
        z_monom = np.array(z.monomials())

        z_linear = [g.index(i) for i in z_monom if i.degree() == 1]

        #z_nlinear = [list(i.variables()) for i in z_monom if len(i.variables()) > 1]
        #z_nlinear = [g.index(z_nlinear[i][j]) for i in range(len(z_nlinear)) for j in range(len(z_nlinear[i]))]
        #z_nlinear = list(set(z_nlinear))


        #linear_new = [i for i in z_linear if i not in (linear + z_nlinear)]
        #z_linear_org = [i for i in z_linear if i not in z_nlinear]


        #One of the linear term variables which is not in the previous linear term has one recovered bit
        model.addConstr(gp.quicksum(m[i,0] for i in linear[loop]) <= 1)    
        #model.addConstr(gp.quicksum(m[i,0] for i in z_linear) <= loop + 1) 



        for mon in z_monom:
            if mon.degree() > 1:
                var = list(mon.variables())
                var = [g.index(i) for i in var]
                inters = [0]*len(linear)
                n = [0]*len(linear)
                for l in range(len(linear)):
                    inters[l] = intersection(var,linear[l])
                    n[l] = len(inters[l])

                C_prev = [n[i] for i in range(loop)] 
                C_current = [n[loop]]
                C_future = [n[i] for i in range(loop+1,keys,1)]

                #Case 1: recovered var in mon and is in linear
                #if sum(C_prev) > 0 and sum(C_current) == 0 and sum(C_future) == 0 :
                #    #if recovery found then guess
                #    model.addConstr((p[count] == 0) >> (gp.quicksum(m[i,0] for i in var) == 0))
                #    model.addConstr((p[count] == 1) >> (gp.quicksum((m[i,1] + m[i,2]) for i in var) >= len(var) - sum(C_prev)))
                #    count += 1

                #Case 2: recovered var in mon and is in z_linear-linear
                if (sum(C_current + C_future) > 0):
                    #if recovery found then fix to zero
                    prev = []
                    for k in range(loop):
                        prev = prev + inters[k]
                    #var_ge = [i for i in var if i not in prev]

                    var_ge = []
                    for k in range(loop,len(linear),1):
                        var_ge = var_ge + inters[k]


                    model.addConstr((q[count] == 0) >> (gp.quicksum(m[i,0] for i in var_ge) == 0))
                    model.addConstr((q[count] == 1) >> (gp.quicksum(m[i,1] for i in var) >= 1))
                    count += 1

                #if (sum(C_prev + C_current + C_future) == 0): 
                #    model.addConstr(gp.quicksum((m[i,0]) for i in var) == 0)
        print("2nd part loop ", loop)
    print("2nd part done")
    import time

    model.params.outputflag = 0
    start = time.time()
    model.optimize()
    end = time.time() - start
    model.params.ObjNumber = 0
    print("Number of Objective function", model.ObjNName)

    print("Keystream = ",keys)
    print("status",model.STATUS)
    
    f = open(filename,'a')
    f.write("\nKeystream = " + str(keys))
    f.write("\nStatus " + str(model.STATUS) + "\n")
    
    if model.STATUS == 2:
        val = model.x
        R = []
        F = []
        G = []
        for i in range(state_len):
            if round(val[i]) == 2:
                R.append(i)
            if round(val[i]) == 3:
                F.append(i)
            if round(val[i]) == 4:
                G.append(i)



        Rec = [g[i] for i in R]
        Fix = [g[i] for i in F]
        Guess = [g[i] for i in G]
        print("Recovery Bit ",Rec)
        print("Fixed Bit ",Fix)
        print("Guessed Bit ",Guess)
        
        f.write(str("Recovery Bit ") + str(Rec) + "\n")
        f.write(str("Fixed Bit ") + str(Fix) + "\n")
        f.write(str("Guessed Bit ") + str(Guess) + "\n")
        
        
        print(len(R),len(F))
        if (len(R) + len(F) + len(G) == state_len) and len(intersection(R,F) + intersection(F,G) + intersection(R,G)) == 0:
            print("Length verified")
            f.write("\nLength Verified\n\n")
    end2 = time.time()-start2
    print("Gurobi Time = ", end)
    print("Overall Time = ", end2)
    
    f.write("\n\nGurobi Time = " + str(end))
    f.write("\nOverall Time = " + str(end2))
    f.close()


    #keys = len(R)
    B = [g[i] for i in range(90)]
    S = [g[90+i] for i in range(31)]

    #########################Fixed bit##########################
    for i in F:
        if i < 90:
            B[i] = 0
        else:
            S[i-90] = 0


    Rec = [g[i] for i in R]
    print("Recovered Bits are ", Rec)
    Fix = [g[i] for i in F]
    print("Fixed Bits are ", Fix)



    R_past = []
    L_rec = []
    cnt = 0
    flag = 1
    for i in range(keys):
        z = keystream(B,S)
        monom = list(z.monomials())
        t_var = list(z.variables())

        R_past = R_past + list(set(L_rec))
        m_rec = []
        for m in monom:
            m_var = list(m.variables())
            if (len(m_var) == 1) and (len(intersection(m_var,Rec)) > 0):
                L_rec = L_rec + m_var

            if (len(m_var) > 1) and (len(intersection(m_var,Rec)) > 0):
                m_rec = m_rec + [m]
        #print("monomials containing recovery bits (degree > 1) ",m_rec)

        for m in m_rec:
            m_var = list(m.variables())
            #Now L_rec contains current and past recovery bit in linear terms
            #Whereas R_past contains only the past recovery bit in linear terms
            if len(intersection(m_var,Rec)) > len(intersection(m_var,R_past)):
                f = open(filename,'a')
                f.write("\nError --- Future recovery bits found\n" + str(m))
                f.close()
                print("Error --- Future recovery bits found")
                print(m)

        L_rec = list(set(L_rec))
        if len(L_rec) != cnt + 1:
            flag = 0
        cnt = len(L_rec)
        print("Recovery bits till now in linear terms ",L_rec)

        B=NFSR2_fw_update(B,S[0])
        S=NFSR1_fw_update(S)

    f = open(filename,'a')
    if flag == 0:
        print("Failed")
        f.write("\nFailed\n")
    else:
        print("Passed")
        f.write("\nPassed\n")
        
        import csv
        with open('Liz_TMDTO_MILP_DATA_copy.csv','a') as f:
            writer = csv.writer(f)
            #writer.writerow(['Number of Recovered bits','Number of Fixed bits','Recovered bits','Fixed bits'])
            writer.writerow([len(R),len(F),Rec,Fix])
    f.close()

NameError: name 'keystream' is not defined

In [46]:
#keys = len(R)
B = [g[i] for i in range(90)]
S = [g[90+i] for i in range(31)]

#########################Fixed bit##########################
for i in F:
    if i < 90:
        B[i] = 0
    else:
        S[i-90] = 0

    
Rec = [g[i] for i in R]
print("Recovered Bits are ", Rec)
Fix = [g[i] for i in F]
print("Fixed Bits are ", Fix)



R_past = []
L_rec = []
cnt = 0
flag = 1
for i in range(keys):
    z = keystream(B,S)
    monom = list(z.monomials())
    t_var = list(z.variables())
    
    R_past = R_past + list(set(L_rec))
    m_rec = []
    for m in monom:
        m_var = list(m.variables())
        if (len(m_var) == 1) and (len(intersection(m_var,Rec)) > 0):
            L_rec = L_rec + m_var
        
        if (len(m_var) > 1) and (len(intersection(m_var,Rec)) > 0):
            m_rec = m_rec + [m]
    #print("monomials containing recovery bits (degree > 1) ",m_rec)
        
    for m in m_rec:
        m_var = list(m.variables())
        #Now L_rec contains current and past recovery bit in linear terms
        #Whereas R_past contains only the past recovery bit in linear terms
        if len(intersection(m_var,Rec)) > len(intersection(m_var,R_past)):
            print("Error --- Future recovery bits found")
            print(m)
            
    L_rec = list(set(L_rec))
    if len(L_rec) != cnt + 1:
        flag = 0
    cnt = len(L_rec)
    #print("Recovery bits till now in linear terms ",L_rec)
        
    B=NFSR2_fw_update(B,S[0])
    S=NFSR1_fw_update(S)
    
if flag == 0:
    print("Failed")
else:
    print("Passed")
    import csv
    with open('Liz_TMDTO_MILP_DATA.csv','a') as f:
        writer = csv.writer(f)
        #writer.writerow(['Number of Recovered bits','Number of Fixed bits','Recovered bits','Fixed bits'])
        writer.writerow([len(R),len(F),Rec,Fix])

Recovered Bits are  [b31, b40, b56, b59, b74, s27, s29]
Fixed Bits are  [b66, b68]
Passed
